In [188]:
from delta_debugging.DD import DD
import difflib

In [189]:
class TestDD(DD):
	def __init__(self):
		DD.__init__(self)
		self.debug_dd = 0
		self.verbose = 0
	def _test(self, deltas):
		# Build input file
		found = []
		for (index, byte) in deltas:
			if byte == "1" or byte == "3":
				found.append(byte)
		ret = self.PASS
		if found.count("1") == 1 and found.count("3") == 1:
			ret = self.FAIL
		print('Testing case {:11}: {}'.format('"' + "".join([x[1] for x in deltas]) + '"', str(ret)))
		return ret



In [204]:
def str_to_deltas(test_input):
    deltas = list(map(lambda x: (x, test_input[x]), range(len(test_input))))
    return deltas

In [190]:
test_input = "12345678"
print('Minimizing input: "{}"'.format(test_input))
# Convert string into the delta format
deltas = list(map(lambda x: (x, test_input[x]), range(len(test_input))))

Minimizing input: "12345678"


In [191]:
mydd = TestDD()
mydd.verbose = 0
mydd.debug_dd = 1
c = mydd.ddmin(deltas)              # Invoke DDMIN
minimal = "".join([x[1] for x in c])
print('Found minimal test case: "{}"'.format(minimal))

dd([(0, '1'), (1, '2'), (2, '3'), (3, '4'), (4, '5'), (5, '6'), (6, '7'), (7, '8')], 2)...
Testing case ""         : PASS
Testing case "12345678" : FAIL
Testing case "5678"     : PASS
Testing case "1234"     : FAIL
dd: reduced to 4
deltas:
[(0, '1'), (1, '2'), (2, '3'), (3, '4')]
Testing case "34"       : PASS
Testing case "12"       : PASS
Testing case "234"      : PASS
Testing case "134"      : FAIL
dd: reduced to 3
deltas:
[(0, '1'), (2, '3'), (3, '4')]
Testing case "14"       : PASS
Testing case "13"       : FAIL
dd: reduced to 2
deltas:
[(0, '1'), (2, '3')]
Testing case "3"        : PASS
Testing case "1"        : PASS
dd: done
dd([(0, '1'), (1, '2'), (2, '3'), (3, '4'), (4, '5'), (5, '6'), (6, '7'), (7, '8')], 2) = [(0, '1'), (2, '3')]
Found minimal test case: "13"


In [192]:
# test_input = "2345678"
# print('Minimizing input: "{}"'.format(test_input))
# # Convert string into the delta format
# deltas = list(map(lambda x: (x, test_input[x]), range(len(test_input))))

# mydd = TestDD()
# mydd.verbose = 1
# mydd.debug_dd = 1
# c = mydd.ddmax(deltas)              # Invoke DDMIN
# minimal = "".join([x[1] for x in c])
# print('Found minimal test case: "{}"'.format(minimal))

In [193]:
mydd = TestDD()
# mydd.verbose = 1
(c, c1, c2) = mydd.dd(deltas)              # Invoke DDMIN
c1 = "".join([x[1] for x in c1])
c2 = "".join([x[1] for x in c2])
c = "".join([x[1] for x in c])
print("The 1-minimal failure-inducing difference is", c)
print(c1, "passes,", c2, "fails")

Testing case "1234"     : FAIL
Testing case "12"       : PASS
Testing case "123"      : FAIL
The 1-minimal failure-inducing difference is 3
12 passes, 123 fails


## String processing
<hr>

In [284]:
def get_mods(string1, string2):
    # Get list of modifications to change string1 into string2
    mods = []
    s = difflib.SequenceMatcher()
    s.set_seqs(string1, string2)
    matching_blocks = s.get_matching_blocks()

    # Traverse the matching blocks and identify insertions
    for i, block in enumerate(matching_blocks):
        # Check fore insertion before first match
        if i == 0 and block.b > 0:
            insert_str = string2[:block.b]
            for char in insert_str:
                mods.append((-1, char, mydd.ADD))
        # Check for insertions between matches
        if i < len(matching_blocks) - 1:
            next_block = matching_blocks[i + 1]
            insert_str = string2[(block.b+1):next_block.b]
            for char in insert_str:
                mods.append((block.a, char, mydd.ADD))

    diff = list(difflib.ndiff(string1, string2))
    remove_idx = 0
    for d in diff:
        if d.startswith("- "):  # Removed character
            mods.append((remove_idx, d[2], mydd.REMOVE))
            remove_idx += 1
        elif d.startswith(" "): # Not removed character
            remove_idx += 1

    return mods


In [291]:
def split_mods(mods):
    prepend = []
    inserted = []
    removed = []
    for pos, char, op in mods:
        if pos == -1:
            prepend.append((pos, char))  # Collect elements with index -1
        elif op == mydd.ADD:
            inserted.append((pos, char))
        elif op == mydd.REMOVE:
            removed.append((pos, char))
    return prepend, inserted, removed

def apply_remove(deltas, removed):
    delta_dict = {idx: char for idx, char in deltas}
    for start_idx, chars in removed:
        for i in range(len(chars)):  # Remove each character from the given start index
            if start_idx + i in delta_dict:
                del delta_dict[start_idx + i]
    deltas = sorted(delta_dict.items())
    return deltas

def apply_insert(deltas, inserted):
    for insert_idx, chars in inserted:
        # Find the index in new_deltas where to insert
        for i, (idx, _) in enumerate(deltas):
            if idx == insert_idx:
                for char in chars:
                    deltas.insert(i + 1, (None, char))  # Use None as a placeholder index
                break
    return deltas

def apply_prepend(deltas, prepend):
    prepend_chars = [(-1, entry[1]) for entry in prepend]
    deltas = prepend_chars + deltas
    return deltas

def apply_mods(deltas, mods):
    prepend, inserted, removed = split_mods(mods)
    deltas = apply_remove(deltas, removed)
    deltas = apply_insert(deltas, inserted)
    deltas = apply_prepend(deltas, prepend)
    deltas = [(i, char) for i, (_, char) in enumerate(deltas)]
    return deltas

In [297]:
string1 = "1222>"
string2 = "<55513>;"
mods = get_mods(string1, string2)
prepend, inserted, removed = split_mods(mods)
# print(mods)
# print(prepend)
# print(inserted)
# print(removed)

[(-1, '<', 'ADD'), (-1, '5', 'ADD'), (-1, '5', 'ADD'), (-1, '5', 'ADD'), (0, '3', 'ADD'), (4, ';', 'ADD'), (1, '2', 'REMOVE'), (2, '2', 'REMOVE'), (3, '2', 'REMOVE')]


In [295]:
deltas1 = str_to_deltas(string1)
deltas2 = str_to_deltas(string2)
# print(deltas1)
# print(deltas2)
# apply_mods(deltas1, mods)
apply_mods(deltas1, mods) == deltas2

True

### Split experiments

In [302]:
def split(mods, n=2):
    subsets = []
    start = 0
    for i in range(n):
        subset = mods[start:int(start + (len(mods) - start) / (n - i))]
        subsets.append(subset)
        start = start + len(subset)
    return subsets

In [305]:
print(split(mods)[0]), print(split(mods)[1])

[(-1, '<', 'ADD'), (-1, '5', 'ADD'), (-1, '5', 'ADD'), (-1, '5', 'ADD')]
[(0, '3', 'ADD'), (4, ';', 'ADD'), (1, '2', 'REMOVE'), (2, '2', 'REMOVE'), (3, '2', 'REMOVE')]


(None, None)

### String combine experiments

In [248]:
def split_mods(mods):
    prepend = []
    inserted = []
    removed = []
    for pos, char, op in mods:
        if pos == -1:
            prepend.append((pos, char))  # Collect elements with index -1
        elif op == mydd.ADD:
            inserted.append((pos, char))
        elif op == mydd.REMOVE:
            removed.append((pos, char))
    return prepend, inserted, removed


In [255]:
def apply_remove(deltas, removed):
    delta_dict = {idx: char for idx, char in deltas}
    for start_idx, chars in removed:
        for i in range(len(chars)):  # Remove each character from the given start index
            if start_idx + i in delta_dict:
                del delta_dict[start_idx + i]
    deltas = sorted(delta_dict.items())
    return deltas

def apply_insert(deltas, inserted):
    for insert_idx, chars in inserted:
        # Find the index in new_deltas where to insert
        for i, (idx, _) in enumerate(deltas):
            if idx == insert_idx:
                for char in chars:
                    deltas.insert(i + 1, (None, char))  # Use None as a placeholder index
                break
    return deltas

def apply_prepend(deltas, prepend):
    if prepend:
        before_idx, chars = prepend[0]  # Always inserting before the first element
        deltas = [(i, char) for i, char in enumerate(chars)] + deltas
    return deltas

def apply_mods(deltas, mods):
    prepend, inserted, removed = split_mods(mods)
    deltas = apply_remove(deltas, removed)
    deltas = apply_insert(deltas, inserted)
    deltas = apply_prepend(deltas, prepend)
    deltas = [(i, char) for i, (_, char) in enumerate(deltas)]
    return deltas

In [257]:
deltas1 = str_to_deltas(string1)
deltas2 = str_to_deltas(string2)
apply_mods(deltas1, all) == deltas2

True

In [ ]:
def apply_deltas(deltas, inserted_before, inserted_between, removed):
    delta_dict = {idx: char for idx, char in deltas}

    for start_idx, chars in removed:
        for i in range(len(chars)):  # Remove each character from the given start index
            if start_idx + i in delta_dict:
                del delta_dict[start_idx + i]

    deltas = sorted(delta_dict.items())

    # Step 3: Insert "Inserted Between" elements while adjusting indices
    for insert_idx, chars in inserted_between:
        # Find the index in new_deltas where to insert
        for i, (idx, _) in enumerate(deltas):
            if idx == insert_idx:
                for char in chars:
                    deltas.insert(i + 1, (None, char))  # Use None as a placeholder index
                break

    if inserted_before:
        before_idx, chars = inserted_before[0]  # Always inserting before the first element
        deltas = [(i, char) for i, char in enumerate(chars)] + deltas

    deltas = [(i, char) for i, (_, char) in enumerate(deltas)]
    return deltas

In [ ]:
deltas1 = str_to_deltas(string1)
deltas2 = str_to_deltas(string2)
apply_deltas(deltas1, inserted_before, inserted_between, removed) == deltas2

### String comparison experiments

In [ ]:
def get_mods(string1, string2):
    # Get list of modifications to change string1 into string2
    mods = []
    s = difflib.SequenceMatcher()
    s.set_seqs(string1, string2)
    matching_blocks = s.get_matching_blocks()

    # Traverse the matching blocks and identify insertions
    for i, block in enumerate(matching_blocks):
        # Check fore insertion before first match
        if i == 0 and block.b > 0:
            insert_str = string2[:block.b]
            mods.append((-1, insert_str, mydd.ADD))
        # Check for insertions between matches
        if i < len(matching_blocks) - 1:
            next_block = matching_blocks[i + 1]
            insert_str = string2[(block.b+1):next_block.b]
            mods.append((block.a, insert_str, mydd.ADD))

    diff = list(difflib.ndiff(string1, string2))
    index = 0
    remove = []
    remove_idx = -1
    for d in diff:
        if d.startswith("- "):  # Removed character
            if remove == []:
                remove_idx = index
            remove.append(d[2])
        elif remove != []:
            mods.append((remove_idx, "".join(remove), mydd.REMOVE))
            remove = []
        if d.startswith(" "): # Not removed character
            index += 1

    return mods

In [ ]:
def string_compare(string1, string2):
    # Initialize lists to track inserted and removed characters
    inserted_before = []
    inserted_between = []
    removed = []

    s = difflib.SequenceMatcher()
    s.set_seqs(string1, string2)
    matching_blocks = s.get_matching_blocks()

    # Traverse the matching blocks and identify insertions
    for i, block in enumerate(matching_blocks):
        # Check fore insertion before first match
        if i == 0 and block.b > 0:
            insert_str = string2[:block.b]
            inserted_before.append((-1, insert_str, mydd.ADD))
        # Check for insertions between matches
        if i < len(matching_blocks) - 1:
            next_block = matching_blocks[i + 1]
            insert_str = string2[(block.b+1):next_block.b]
            inserted_between.append((block.a, insert_str, mydd.ADD))

    diff = list(difflib.ndiff(string1, string2))
    index = 0
    remove = []
    remove_idx = -1
    for d in diff:
        if d.startswith("- "):  # Removed character
            if remove == []:
                remove_idx = index
            remove.append(d[2])
        elif remove != []:
            removed.append((remove_idx, "".join(remove), mydd.REMOVE))
            remove = []
        if d.startswith(" "): # Not removed character
            index += 1
    print("Inserted before:", inserted_before)
    print("Inserted between:", inserted_between)
    print("Removed:", removed)
    print("All:", removed + inserted_before + inserted_between)
    all = removed + inserted_before + inserted_between

    return inserted_before, inserted_between, removed, all


In [258]:
string1 = "1222>"
string2 = "<55513>;"
inserted_before, inserted_between, removed, all = string_compare(string1, string2)

Inserted before: [(-1, '<555', 'ADD')]
Inserted between: [(0, '3', 'ADD'), (4, ';', 'ADD')]
Removed: [(1, '222', 'REMOVE')]
All: [(1, '222', 'REMOVE'), (-1, '<555', 'ADD'), (0, '3', 'ADD'), (4, ';', 'ADD')]


In [97]:
diff = list(difflib.ndiff(pass_input, fail_input)); diff

['  0', '  0', '  2', '  2', '- 6', '- 6', '+ 8', '+ 8', '+ 1', '+ 3']

In [153]:
import difflib

def compute_detailed_deltas(string1, string2):
    """
    Computes four types of deltas:
    - ADD_BEFORE: Characters that must be inserted before existing ones.
    - ADD_BETWEEN: Characters inserted between unchanged characters.
    - ADD_AFTER: Characters that must be inserted after the original content.
    - REMOVE: Characters that must be removed.

    Args:
    - string1 (str): The original string.
    - string2 (str): The target string.

    Returns:
    - dict: A dictionary with 'ADD_BEFORE', 'ADD_BETWEEN', 'ADD_AFTER', 'REMOVE' operations.
    """
    diff = list(difflib.ndiff(string1, string2))
    last_char = string2[-1]
    print(diff)

    add_before = []
    add_between = []
    add_after = []
    remove = []

    seen_first_match = False  # Track if we've seen the first unchanged character
    seen_last_match = False   # Track if we've passed all unchanged characters

    for d in diff:
        if d.startswith("- "):  # Removed character
            remove.append(d[2])
        elif d.startswith("+ "):  # Added character
            if not seen_first_match:  
                add_before.append(d[2])  # Happens before any common character
            elif seen_last_match:  
                add_after.append(d[2])  # Happens after all common characters
            else:
                add_between.append(d[2])  # Inserted inside existing content
        else:  # Unchanged character
            seen_first_match = True
        if seen_first_match and not d.startswith(" "):
            seen_last_match = True  # We've moved past the last matching sequence

    return {
        "ADD_BEFORE": add_before,
        "ADD_BETWEEN": add_between,
        "ADD_AFTER": add_after,
        "REMOVE": remove
    }

# Example usage:

string1 = "1222>"
string2 = "<55513>;"

deltas = compute_detailed_deltas(string1, string2)
print("ADD_BEFORE:", deltas["ADD_BEFORE"])   # Expected: ['s']
print("ADD_BETWEEN:", deltas["ADD_BETWEEN"]) # Expected: ['i']
print("ADD_AFTER:", deltas["ADD_AFTER"])     # Expected: ['g']
print("REMOVE:", deltas["REMOVE"])           # Expected: ['k', 'e']


['+ <', '+ 5', '+ 5', '+ 5', '  1', '+ 3', '- 2', '- 2', '- 2', '  >', '+ ;']
ADD_BEFORE: ['<', '5', '5', '5']
ADD_BETWEEN: ['3']
ADD_AFTER: [';']
REMOVE: ['2', '2', '2']


In [161]:
from deepdiff import DeepDiff

In [174]:
diff = DeepDiff(list(string1), list(string2))
diff.get("dictionary_item_removed", [])

[]

In [175]:
print(diff)

{'values_changed': {'root[0]': {'new_value': '<', 'old_value': '1'}, 'root[1]': {'new_value': '5', 'old_value': '2'}, 'root[2]': {'new_value': '5', 'old_value': '2'}, 'root[3]': {'new_value': '5', 'old_value': '2'}, 'root[4]': {'new_value': '1', 'old_value': '>'}}, 'iterable_item_added': {'root[5]': '3', 'root[6]': '>', 'root[7]': ';'}}


In [154]:
def isjunk(string):
    "Return True if we don't care about this string"
    return string == ' '


In [155]:
s = difflib.SequenceMatcher(isjunk)
s.set_seqs(string1, string2)

In [156]:
matching_blocks = s.get_matching_blocks(); print(matching_blocks)

[Match(a=0, b=4, size=1), Match(a=4, b=6, size=1), Match(a=5, b=8, size=0)]


In [177]:
# Initialize lists to track inserted and removed characters
# insert_before = []
inserts = []
# insert_after = []
inserted_before = []
inserted_between = []
removed = []

# Traverse the matching blocks and identify insertions
for i, block in enumerate(matching_blocks):
    # Check fore insertion before first match
    if i == 0 and block.b > 0:
        insert_str = string2[:block.b]
        inserted_before.append((insert_str, -1))
    # Check for insertions between matches
    if i < len(matching_blocks) - 1:
        next_block = matching_blocks[i + 1]
        insert_str = string2[(block.b+1):next_block.b]
        inserted_between.append((insert_str, block.a))

diff = list(difflib.ndiff(string1, string2))
index = 0
remove = []
remove_idx = -1
for d in diff:
    if d.startswith("- "):  # Removed character
        if remove == []:
            remove_idx = index
        remove.append(d[2])
    elif remove != []:
        removed.append(("".join(remove), remove_idx))
        remove = []
    if d.startswith(" "): # Not removed character
        index += 1

# For removed characters, check those in string1 that are not in string2
# removed = [string1[i] for i in range(len(string1)) if string1[i] not in string2]

# Output the results
print("Inserted before:", inserted_before)
print("Inserted between:", inserted_between)
print("Removed:", removed)

Inserted before: [('<555', -1)]
Inserted between: [('3', 0), (';', 4)]
Removed: [('222', 1)]


In [179]:
# print(list(s.get_opcodes()))
for (
        opcode,
        before_start, before_end,
        after_start, after_end
) in s.get_opcodes():
    if opcode == 'equal':
        # We don't care.
        continue
    print ("%7s '%s' -> '%s'" % (
            opcode,
            string1[before_start:before_end],
            string2[after_start:after_end],
) )

 insert '' -> '<555'
replace '222' -> '3'
 insert '' -> ';'


In [ ]:
delta_add_before = []
delta_remove = []
for d in diff:
        if d.startswith("- "):  # Removal
            remove.append(d[2])
            prev_char = d[2]  # Removed character
        elif d.startswith("+ "):  # Addition
            if prev_char is None:  # At the beginning
                add_before.append(d[2])
            else:  # After some existing character
                add_after.append(d[2])
            prev_char = None  # Reset since an addition follows a valid character
        else:  # Unchanged character
            prev_char = d[2]

In [88]:
delta_add = []
delta_remove = []
for i,s in enumerate(difflib.ndiff(pass_input, fail_input)):
    if s[0]=='-':
        delta_remove.append(s[-1])
    elif s[0]=='+': 
        delta_add.append(s[-1])
print(f'delta_add: {delta_add}, delta_remove: {delta_remove}')

delta_add: ['8', '8', '1', '3'], delta_remove: ['2', '4', '6', '6']


In [87]:
for a,b in [(pass_input, fail_input)]:     
    print('{} => {}'.format(a,b))  
    for i,s in enumerate(difflib.ndiff(a, b)):
        if s[0]==' ': 'Nothing'
        elif s[0]=='-':
            print(u'Delete "{}" from position {}'.format(s[-1],i))
        elif s[0]=='+':
            print(u'Add "{}" to position {}'.format(s[-1],i))    
    print()      

2466 => 8813
Delete "2" from position 0
Delete "4" from position 1
Delete "6" from position 2
Delete "6" from position 3
Add "8" to position 4
Add "8" to position 5
Add "1" to position 6
Add "3" to position 7



## Others

In [ ]:
pass_input = "002266"
fail_input = "00228813"
# c1 = list(map(lambda x: (x, pass_input[x]), range(len(pass_input))))
# c2 = list(map(lambda x: (x, fail_input[x]), range(len(fail_input))))
# mydd._DD__listunion(c1, c2)

In [45]:
all_input = pass_input + fail_input
c = list(map(lambda x: (x, all_input[x]), range(len(all_input))))
c1 = c[:3]
c2 = c[3:]

In [46]:
print(mydd.test(c1)); print(mydd.test(c2))

PASS
Testing case "8813"     : FAIL
FAIL


In [47]:
c = c2
cs = mydd.split(c, 2)

In [48]:
cs

[[(3, '8'), (4, '8')], [(5, '1'), (6, '3')]]

In [53]:
i=0
mydd = TestDD()
mydd.test_and_resolve(cs[i], c1, c, mydd.REMOVE)

Testing case "24688"    : PASS


('PASS', [(3, '8'), (4, '8')])

In [58]:
mydd.outcome_cache.tail

{(0, '2'): <delta_debugging.DD.OutcomeCache at 0x11bb73ad0>}

In [59]:
list(map(lambda x: (x, all_input[x], mydd.ADD), range(len(all_input))))

[(0, '2', 'ADD'),
 (1, '4', 'ADD'),
 (2, '6', 'ADD'),
 (3, '8', 'ADD'),
 (4, '8', 'ADD'),
 (5, '1', 'ADD'),
 (6, '3', 'ADD')]

In [60]:
s1 = {}
for delta in c1:
    s1[delta] = 1

In [61]:
s1

{(0, '2'): 1, (1, '4'): 1, (2, '6'): 1}

In [64]:
delta = (0, '2', mydd.REMOVE)

In [66]:
s1[delta[:2]] -=1

In [67]:
s1

{(0, '2'): 0, (1, '4'): 1, (2, '6'): 1}

In [73]:
c1.remove(delta[:2])

In [74]:
c1

[(1, '4'), (2, '6')]